In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# SETTINGS
learning_rate = 1e-2
torch.manual_seed(42)

device = 'cpu'
if torch.cuda.is_available():
	device = 'cuda'

print(device)

cpu


In [3]:
mnist_train = torchvision.datasets.MNIST('.', train=True, download=True)
mnist_test = torchvision.datasets.MNIST('.', train=False, download=True)

data_train = mnist_train.data.to(device).float()
data_test = mnist_test.data.to(device).float()

target_train = mnist_train.targets.to(device)
target_test = mnist_test.targets.to(device)

print(data_train.shape, data_test.shape)
print(data_train.dtype, data_test.dtype)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.float32 torch.float32


In [4]:
class Model(nn.Module):
	def __init__(self):
		super().__init__()
		self.net = nn.Sequential(
			nn.Linear(784, 800),
			nn.ReLU(),
			nn.LayerNorm(800),
			nn.Dropout(0.2),
			nn.Linear(800, 10),
		)
		
	def forward(self, x, y=None):
		r = x.view(-1, 784)
		r = self.net(r)
		if y is None:
			loss = None
		else:
			loss = F.cross_entropy(r, y)
		return r, loss

model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
estimation_iters = 4
estimation_batch = 256

def get_split(split):
	data = data_train if split == 'train' else data_test
	target = target_train if split == 'train' else target_test
	return data, target

def get_batch(batch_size, split):
	data, target = get_split(split)
	ix = torch.randint(0, len(data), (batch_size,))
	x = torch.stack([data[i] for i in ix])
	y = torch.stack([target[i] for i in ix])
	return x, y

@torch.no_grad()
def estimate_loss(split = None):
	if split == None:
		return estimate_loss('train'), estimate_loss('test')
	losses = torch.zeros(estimation_iters)
	for i in range(estimation_iters):
		x, y = get_batch(estimation_batch, split)
		logits, loss = model.forward(x, y)
		losses[i] = loss
	return losses.mean().item()

@torch.no_grad()
def total_loss(split = None):
	total_estimation_batch = 1 # ?
	if split == None:
		return total_loss('train'), total_loss('test')
	data, target = get_split(split)
	losses = torch.zeros(len(data) // total_estimation_batch)
	for i in range(len(data) // total_estimation_batch):
		lower = i * total_estimation_batch
		upper = min((i + 1) * total_estimation_batch, len(data))
		x = data[lower:upper]
		y = target[lower:upper]
		logits, loss = model.forward(x, y)
		losses[i] = loss
	return losses.mean().item()

In [6]:
training_iters = 1000
training_batch = 64
eval_interval = 10
batch_losses = []
train_losses = []
test_losses = []
for iter in range(training_iters):
	if (iter % eval_interval) == 0:
		loss_train, loss_test = estimate_loss()
		test_losses.append(loss_test)
		train_losses.append(loss_train)
		print(f'iter {iter}: loss_train={loss_train}, loss_test={loss_test}')

	x, y = get_batch(training_batch, 'train')
	logits, loss = model(x, y)
	optimizer.zero_grad()
	loss.backward()
	optimizer.step()
	batch_losses.append(loss.item())

iter 0: loss_train=2.4548287391662598, loss_test=2.396651029586792
iter 10: loss_train=1.689379096031189, loss_test=1.7093123197555542
iter 20: loss_train=0.5847361087799072, loss_test=0.6152201890945435
iter 30: loss_train=0.6434414386749268, loss_test=0.6174726486206055
iter 40: loss_train=0.5337696671485901, loss_test=0.5469639897346497
iter 50: loss_train=0.38543641567230225, loss_test=0.387143611907959
iter 60: loss_train=0.34983181953430176, loss_test=0.4302836060523987
iter 70: loss_train=0.3249760568141937, loss_test=0.31451690196990967
iter 80: loss_train=0.3804546594619751, loss_test=0.3252601623535156
iter 90: loss_train=0.3583514988422394, loss_test=0.3323138356208801
iter 100: loss_train=0.3735160827636719, loss_test=0.3356773257255554
iter 110: loss_train=0.26930326223373413, loss_test=0.311576783657074
iter 120: loss_train=0.29076698422431946, loss_test=0.2708670198917389
iter 130: loss_train=0.31423336267471313, loss_test=0.3286907374858856
iter 140: loss_train=0.286610

In [7]:
accuracy_max = 0
accuracy_mno = 0
for i in range(len(data_test)):
	logits, _ = model(data_test[i])
	result_max = logits.argmax()
	result_mno = torch.multinomial(F.softmax(logits, dim=-1), 1)
	if result_max == target_test[i]:
		accuracy_max += 1
	if result_mno == target_test[i]:
		accuracy_mno += 1

accuracy_max /= len(data_test)
accuracy_mno /= len(data_test)

In [8]:
loss = total_loss('test')

In [9]:
import math
print(accuracy_max, accuracy_mno, loss, math.exp(-loss))

0.9517 0.9378 0.15387371182441711 0.8573802911674455


In [10]:
def predict_one(index):
	y, _ = model(data_test[index])
	s = torch.softmax(y, dim=-1)
	m = torch.argmax(y)
	for v, i in zip(s[0], range(10)):
		print("{}: {:.3f}".format(i, v))
	print(f'max={m}, target: {target_test[index]}')
	return mnist_test[index][0]

import random
predict_one(random.randint(0, data_test.shape[0]))

0: 0.000
1: 0.000
2: 0.000
3: 0.001
4: 0.000
5: 0.993
6: 0.000
7: 0.000
8: 0.007
9: 0.000
max=5, target: 5
